## Ingest qualifying folder

1. Read the JSON file using the spark dataframe reader

In [0]:
from pyspark.sql.functions import current_timestamp, col, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
qualifying_schema = StructType(
    fields=[
        StructField("qualifyId", IntegerType(), False),
        StructField("raceId", IntegerType(), True),
        StructField("driverId", IntegerType(), True),
        StructField("constructorId", IntegerType(), True),
        StructField("number", IntegerType(), True),
        StructField("position", IntegerType(), True),
        StructField("q1", StringType(), True),
        StructField("q2", StringType(), True),
        StructField("q3", StringType(), True)
    ]
)

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
qualifying_df = spark.read \
  .option("multiline", True) \
  .schema(qualifying_schema) \
  .json(f'{raw_folder_path}/qualifying')
# display(qualifying_df)
# qualifying_df.count()

2. Process Dataframe

In [0]:
qualifying_final_df = qualifying_df.withColumnRenamed("qualifyingId", "qualifying_id") \
    .withColumnRenamed("raceId", "race_id") \
    .withColumnRenamed("DriverId", "driver_id") \
    .withColumnRenamed("constructorId", "constructor_id") \
    .withColumn("data_source", lit(v_data_source))
qualifying_df = add_ingestion_date(qualifying_final_df)
# display(qualifying_final_df)

3. Write data to datalake as parquet

In [0]:
#qualifying_final_df.write.mode("overwrite").parquet("/mnt/formula1jf/processed/qualifying")
# database
qualifying_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.qualifying")

In [0]:
# %fs
# ls mnt/formula1jf/processed/qualifying

In [0]:
dbutils.notebook.exit("Success")